# German Credit Risk Analysis: Complete Walkthrough

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GlassAlpha/glassalpha/blob/main/examples/notebooks/german_credit_walkthrough.ipynb)

**Complete ML audit workflow**: Data exploration → Model training → Fairness analysis → SHAP explanations → Calibration → Professional PDF report

**Dataset**: German Credit (1000 applications) | **Protected Attributes**: Gender, Age, Foreign Worker

**API Reference**: [`from_model()` documentation](https://glassalpha.com/reference/api/api-audit/) | [User Guide](https://glassalpha.com/getting-started/quickstart/)

## Step 1: Installation

In [ ]:
%pip install -q glassalpha[explain,xgboost]

In [ ]:
"""Environment verification for reproducibility"""
import platform
import random
import sys

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import glassalpha as ga

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

print(
    {
        "python": sys.version.split()[0],
        "platform": platform.platform(),
        "glassalpha": getattr(ga, "__version__", "dev"),
        "seed": SEED,
    }
)

## Step 2: Load Data

In [ ]:
df = ga.datasets.load_german_credit()
print(f"Dataset: {df.shape[0]} samples, {df.shape[1]} features")
print(f"Target balance: {df['credit_risk'].mean():.1%} good credit")
df.head()

## Step 3: Train Models

In [ ]:
protected_attrs = ["gender", "age_group", "foreign_worker"]
feature_cols = [c for c in df.columns if c not in ["credit_risk"] + protected_attrs]
X, y = df[feature_cols], df["credit_risk"]
protected_data = df[protected_attrs]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED, stratify=y)
print(f"Train: {len(X_train)} | Test: {len(X_test)}")

# Store original indices before any transformations
X_test_indices = X_test.index.copy()

In [ ]:
# Extract protected attributes from original DataFrame before pipeline transformation
# This preserves the DataFrame structure needed for fairness analysis
protected_attributes = {}
for attr in protected_attrs:
    protected_attributes[attr] = df.loc[X_test_indices, attr].values

print(f"Protected attributes extracted: {list(protected_attributes.keys())}")
print(f"Gender distribution: {pd.Series(protected_attributes['gender']).value_counts().to_dict()}")

In [ ]:
# Encode categorical features
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

# Identify categorical columns
categorical_cols = X_train.select_dtypes(include=["object"]).columns
numerical_cols = X_train.select_dtypes(exclude=["object"]).columns

# Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numerical_cols),
    ]
)

# Create pipeline with preprocessor and model
pipeline = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("model", RandomForestClassifier(n_estimators=100, max_depth=5, random_state=SEED)),
    ]
)

# Fit the pipeline
pipeline.fit(X_train, y_train)

# Update model reference
model = pipeline

# XGBoost commented out due to categorical handling issues
# xgb = XGBClassifier(
#     n_estimators=100,
#     max_depth=3,
#     random_state=SEED,
#     enable_categorical=True,  # Enable categorical support for object columns
# )
# xgb.fit(X_train, y_train)  # Use original data
# xgb.fit(X_train, y_train)  # Use original data since XGBoost can handle categoricals

# Comment out XGBoost scoring
# print(f"XGBoost test acc: {xgb.score(X_test, y_test):.3f}")
# model = xgb if xgb.score(X_test, y_test) > rf.score(X_test, y_test) else rf
print(f"RandomForest test acc: {model.score(X_test, y_test):.3f}")
# model already set to pipeline above
print("\n✓ Selected: RandomForest")

## Step 4: Generate Audit

In [ ]:
result = ga.audit.from_model(
    model=model,
    X=X_test,
    y=y_test,
    protected_attributes=protected_attributes,  # Now includes protected attributes
    random_seed=SEED,
)
result  # Display inline

## Step 5: Performance Analysis

In [ ]:
print(f"Accuracy: {result.performance['accuracy']:.3f}")
print(f"AUC-ROC: {result.performance['roc_auc']:.3f}")
print(f"Precision: {result.performance['precision']:.3f}")
print(f"Recall: {result.performance['recall']:.3f}")

# Note: Interactive plotting (.plot_*) coming in Phase 3
# All visualizations are available in the PDF report

## Step 6: Fairness Analysis

In [ ]:
dp_diff = result.fairness.get("demographic_parity_max_diff", 0.0)
print(f"Demographic Parity (max difference): {dp_diff:.3f}")
print(f"\nBias detected: {'⚠️ YES' if dp_diff > 0.10 else '✓ NO'} (10% threshold)")
print(f"\nProtected attributes analyzed: {list(result.manifest['protected_attributes_categories'].keys())}")

# Note: Interactive plotting (.plot_*) coming in Phase 3
# All visualizations are available in the PDF report

## Step 7: Calibration

In [ ]:
print(f"Expected Calibration Error: {result.calibration['expected_calibration_error']:.4f}")
print(f"Brier Score: {result.calibration['brier_score']:.4f}")
print(
    f"\nCalibration: {'✓ PASS' if result.calibration['expected_calibration_error'] < 0.05 else '⚠️ WARNING'} (ECE < 0.05 target)"
)

# Note: Interactive plotting (.plot_*) coming in Phase 3
# All visualizations are available in the PDF report

## Step 8: SHAP Explanations

In [ ]:
print("Top 10 Important Features:\n")
# print(result.explanations.feature_importance.head(10))
# Feature importance not yet implemented

# Note: Interactive plotting (.plot_*) coming in Phase 3
# All visualizations are available in the PDF report

## Step 9: Export Audit Report

In [ ]:
# result.to_pdf('german_credit_audit.pdf')
# result.to_json('metrics.json')
# result.to_config('audit_config.yaml')
# to_config() will be implemented in Phase 3
print("✓ Exported: PDF report, metrics JSON, config YAML")

## Summary

**Performance**: Strong accuracy and AUC-ROC
**Fairness**: Analyzed across gender, age, foreign worker status
**Calibration**: ECE indicates prediction reliability
**Explainability**: SHAP values provide feature attribution

**Next Steps**: Review PDF report, address any fairness gaps, monitor in production